In [1]:
from sympy import *
from sympy.printing import latex




In [2]:
#Initialize symbols
x, y, u, v = symbols('x y u v', real=True)
xe, ye, ue, ve = symbols('xe ye ue ve', real=True)
r, s = symbols('r s', real=True)
P = symbols('P', real=True, commutative=True, cls=Function)
Q = symbols('Q', real=True, commutative=True, cls=Function)

#Define differentials
dx = lambda f : diff(f,x)
dy = lambda f : diff(f,y)
du = lambda f : diff(f,u)
dv = lambda f : diff(f,v)


#Define convenient substitutions
switchzw = ([(x,ue), (y,ve), (u,x), (v,y), (ue,u), (ve,v)])
allz = ([(u,x), (v,y)])
allw = ([(x,u), (y,v)])
subsr = ([((u-x)**2+(v-y)**2,r**2)])
subsr.append(((-u+x)**2+(-v+y)**2,r**2))
subsP0 = ([(P(0),1)])


We first define the covariance kernel $K(z,w)= \mathbb{E}[F(z) \overline{F(w)}]$ and all derivatives with respect to real and imaginary parts of $z=x+iy$ and $w=u+iv$

In [3]:
#Define kernel
K = P((x-u)**2+(y-v)**2)*exp(I*(-x*v+y*u))
K = K.simplify()
K

P((u - x)**2 + (v - y)**2)*exp(I*(u*y - v*x))

In [4]:
Ke = K
Ke = Ke.simplify()
#
dxK = dx(K)
dxKe = dxK
dxKe = dxKe.simplify()
#
dyK = dy(K)
dyKe = dyK
dyKe = dyKe.simplify()
#
duK = du(K)
duKe = duK
duKe = duKe.simplify()
#
dvK = dv(K)
dvKe = dvK
dvKe = dvKe.simplify()
#
dxduKe = du(dxK)
dxduKe = dxduKe.simplify()
#
dyduKe = du(dyK)
dyduKe = dyduKe.simplify()
#
dxdvKe = dv(dxK)
dxdvKe = dxdvKe.simplify()
#
dydvKe = dv(dyK)
dydvKe = dydvKe.simplify()



Define derivatives of $P$ for later use

In [5]:
Pprimer = dx(P((u-x)**2+(v-y)**2))/(2*x-2*u)
Pprimer

Subs(Derivative(P(_xi_1), _xi_1), _xi_1, (u - x)**2 + (v - y)**2)

In [6]:
Ppe = Pprimer.subs(subsr)
Ppe

Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)

In [7]:
Pprimer2 = dx(Pprimer)/(2*x-2*u)
Pprimer2

Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, (u - x)**2 + (v - y)**2)

In [8]:
Pppe = Pprimer2.subs(subsr)
Pppe

Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2)

Simplify derivatives of the kernel by collecting $P$ and $P'$

In [9]:
dxduKe = dxduKe.collect(Pprimer)
dyduKe = dyduKe.collect(Pprimer)
dxdvKe = dxdvKe.collect(Pprimer)
dydvKe = dydvKe.collect(Pprimer)
dxduKe = dxduKe.collect(P((u-x)**2+(v-y)**2))
dyduKe = dyduKe.collect(P((u-x)**2+(v-y)**2))
dxdvKe = dxdvKe.collect(P((u-x)**2+(v-y)**2))
dydvKe = dydvKe.collect(P((u-x)**2+(v-y)**2))

Define the matrix $\Gamma(z,w)$

In [10]:
g11 = Ke
g12 = duKe
g13 = dvKe
g21 = dxKe
g22 = dxduKe
g23 = dxdvKe
g31 = dyKe
g32 = dyduKe
g33 = dydvKe
G = Matrix([[g11,g12, g13],[g21,g22,g23],[g31,g32,g33]])
G

Matrix([
[                                                                                     P((u - x)**2 + (v - y)**2)*exp(I*(u*y - v*x)),                                                                                                                          (I*y*P((u - x)**2 + (v - y)**2) + 2*(u - x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, (u - x)**2 + (v - y)**2))*exp(I*(u*y - v*x)),                                                                                                                          (-I*x*P((u - x)**2 + (v - y)**2) + 2*(v - y)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, (u - x)**2 + (v - y)**2))*exp(I*(u*y - v*x))],
[-(I*v*P((u - x)**2 + (v - y)**2) + 2*(u - x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, (u - x)**2 + (v - y)**2))*exp(I*(u*y - v*x)),         (v*y*P((u - x)**2 + (v - y)**2) - 4*(u - x)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, (u - x)**2 + (v - y)**2) + (-2*I*v*(u - x) - 2*I*y*(u - x) - 2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, (u - x)**2 + 

Define the covariance matrix of $\big(F(z), F^{(1,0)}(z), F^{(0,1)}(z), F(w), F^{(1,0)}(w), F^{(0,1)}(w)\big)$ by using the blocks $\Gamma(z,z)$, $\Gamma(z,w)$, $\Gamma(w,z)$, and $\Gamma(w,w)$

In [11]:
Gbig = Matrix(BlockMatrix([[G.subs(allz), G],[G.subs(switchzw),G.subs(allw)]]))
Gbig = Gbig.subs(subsP0)
Gbig = Gbig.subs(subsr)
Gbig

Matrix([
[                                                                                             1,                                                                                                                                                                                                     I*y,                                                                                                                                                                                                   -I*x,                                                                   P(r**2)*exp(I*(u*y - v*x)),                                                                                                       (I*y*P(r**2) + 2*(u - x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x)),                                                                                                       (-I*x*P(r**2) + 2*(v - y)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x))],
[      

To calculate the covariance matrix $\Omega=A-B C^{-1} B^*$, separate the big matrix above into blocks

In [12]:
A = Matrix(BlockMatrix([[Gbig[1:3,1:3], Gbig[1:3,4:6]],[Gbig[4:6,1:3],Gbig[4:6,4:6]]]))
A

Matrix([
[                                                                                                                                                   y**2 - 2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, 0),                                                                                                                                                                                               -x*y - I,         (v*y*P(r**2) - 4*(u - x)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) + (-2*I*v*(u - x) - 2*I*y*(u - x) - 2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x)), (-4*(u - x)*(v - y)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) + (-v*x - I)*P(r**2) + (-2*I*v*(v - y) + 2*I*x*(u - x))*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x))],
[                                                                                                                                                                                               -x*y 

In [13]:
B = Matrix(BlockMatrix([[Gbig[1:3,0:1], Gbig[1:3,3:4]],[Gbig[4:6,0:1],Gbig[4:6,3:4]]]))
B

Matrix([
[                                                                                          -I*y, -(I*v*P(r**2) + 2*(u - x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x))],
[                                                                                           I*x,  (I*u*P(r**2) - 2*(v - y)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x))],
[-(I*y*P(r**2) + 2*(-u + x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(-u*y + v*x)),                                                                                         -I*v],
[ (I*x*P(r**2) - 2*(-v + y)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(-u*y + v*x)),                                                                                          I*u]])

In [14]:
Bconj = Matrix(BlockMatrix([[Gbig[0:1,1:3], Gbig[0:1,4:6]],[Gbig[3:4,1:3],Gbig[3:4,4:6]]]))
Bconj

Matrix([
[                                                                                          I*y,                                                                                           -I*x, (I*y*P(r**2) + 2*(u - x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x)), (-I*x*P(r**2) + 2*(v - y)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x))],
[(I*v*P(r**2) + 2*(-u + x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(-u*y + v*x)), (-I*u*P(r**2) + 2*(-v + y)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(-u*y + v*x)),                                                                                         I*v,                                                                                         -I*u]])

In [15]:
C = Matrix(BlockMatrix([[Gbig[0:1,0:1], Gbig[0:1,3:4]],[Gbig[3:4,0:1],Gbig[3:4,3:4]]]))
C

Matrix([
[                          1, P(r**2)*exp(I*(u*y - v*x))],
[P(r**2)*exp(I*(-u*y + v*x)),                          1]])

In [16]:
Omega = A - B*C.inv()*Bconj
Omega

Matrix([
[                                                                                                                                                        y**2 - I*y*(-I*y/(-P(r**2)**2*exp(I*(-u*y + v*x))*exp(I*(u*y - v*x)) + 1) + (I*v*P(r**2) + 2*(u - x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*P(r**2)*exp(I*(-u*y + v*x))*exp(I*(u*y - v*x))/(-P(r**2)**2*exp(I*(-u*y + v*x))*exp(I*(u*y - v*x)) + 1)) - (I*v*P(r**2) + 2*(-u + x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*(I*y*P(r**2)*exp(I*(u*y - v*x))/(-P(r**2)**2*exp(I*(-u*y + v*x))*exp(I*(u*y - v*x)) + 1) - (I*v*P(r**2) + 2*(u - x)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2))*exp(I*(u*y - v*x))/(-P(r**2)**2*exp(I*(-u*y + v*x))*exp(I*(u*y - v*x)) + 1))*exp(I*(-u*y + v*x)) - 2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, 0),                                                                                                                                                                                                   -x*y

Simplify single elements of $\Omega$. Note that Python starts indexing rows and columns from 0.

We start with the imaginary part of $\Omega_{1,2}$.

In [17]:
O01im = (-I*Omega[0,1]).simplify().expand().subs([(I,0)])
O01im = (O01im+1).simplify()-1
O01im = O01im.subs([(((u-x)**2+(v-y)**2).expand(),r**2)])
O01im

2*r**2*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)/(P(r**2)**2 - 1) - 1

Next, the imaginary part of $\Omega_{3,4}$ follows

In [18]:
O23im = (-I*Omega[2,3]).simplify().expand().subs([(I,0)])
O23im = (O23im+1).simplify()-1
O23im = O23im.subs([(((u-x)**2+(v-y)**2).expand(),r**2)])
O23im

2*r**2*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)/(P(r**2)**2 - 1) - 1

To derive $\Omega_{1,4}$ and $\Omega_{3,2}$, we first simplify without the exponential factor.

In [19]:
O03re = (Omega[0,3]*exp(-I*(u*y-x*v))).simplify().expand().subs([(I,0)]).simplify().factor()
O03re = O03re*exp(I*(u*y-x*v))
O03re

-(-u + x)*(-v + y)*(4*P(r**2)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 4*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + P(r**2) - 4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2))*exp(I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

In [20]:
O03im = (-I*Omega[0,3]*exp(-I*(u*y-x*v))).simplify().expand().subs([(I,0)]).factor()
O03im = O03im.collect(Ppe)
O03im = O03im.subs([(2*((u-x)**2+(v-y)**2).expand(),2*r**2)])
O03im = O03im*exp(I*(u*y-x*v))
O03im

(2*r**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - P(r**2)**3 + P(r**2))*exp(I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

Thus, $\Omega_{1,4}$ is

In [21]:
O03 = (O03re + I * O03im).simplify()
O03

(-(u - x)*(v - y)*(4*P(r**2)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 4*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + P(r**2) - 4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2)) + I*(2*r**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - P(r**2)**3 + P(r**2)))*exp(I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

In [22]:
O21re = (Omega[2,1]*exp(I*(u*y-x*v))).simplify().expand().subs([(I,0)]).factor()
O21re = O21re*exp(-I*(u*y-x*v))
O21re

-(-u + x)*(-v + y)*(4*P(r**2)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 4*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + P(r**2) - 4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2))*exp(-I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

In [23]:
O21im = (-I*Omega[2,1]*exp(I*(u*y-x*v))).simplify().expand().subs([(I,0)]).factor()
O21im = O21im.collect(Ppe)
O21im = O21im.subs([(2*((u-x)**2+(v-y)**2).expand(),2*r**2)])
O21im = O21im*exp(-I*(u*y-x*v))
O21im

(2*r**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - P(r**2)**3 + P(r**2))*exp(-I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

Thus, $\Omega_{3,2}$ is

In [24]:
O21 = (O21re + I * O21im).simplify()
O21

(-(u - x)*(v - y)*(4*P(r**2)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 4*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + P(r**2) - 4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2)) + I*(2*r**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - P(r**2)**3 + P(r**2)))*exp(-I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

Hence, the real part of the product $\Omega_{1,4}\Omega_{3,2}$ is

In [25]:
O0321re = (O03re*O21re-O03im*O21im).simplify()
O0321re

((u - x)**2*(v - y)**2*(4*P(r**2)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 4*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + P(r**2) - 4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2))**2 - (2*r**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - P(r**2)**3 + P(r**2))**2)/((P(r**2) - 1)**2*(P(r**2) + 1)**2)

Substitute $Q = 4P^2P''-4PP'^2-4P''$

In [26]:
O0321re = O0321re.subs([(4*P(r**2)**2*Pppe-4*P(r**2)*Ppe**2-4*Pppe,Q(r**2))])
O0321re

((u - x)**2*(v - y)**2*(P(r**2) + Q(r**2))**2 - (2*r**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - P(r**2)**3 + P(r**2))**2)/((P(r**2) - 1)**2*(P(r**2) + 1)**2)

Continuing with $\Omega_{2,4}$

In [27]:
O13 = (Omega[1,3]*exp(-I*(u*y-x*v))).simplify().expand().factor()
O13 = O13.collect(y*v).collect(y**2).collect(v**2)
O13 = O13.simplify()
O13 = O13.collect(-P(r**2)**2*Pppe+P(r**2)*Ppe**2+Pppe)
O13 = O13.collect(P(r**2))
O13 = O13.collect(Ppe)
O13 = O13.subs([(P(r**2)**2*Pppe-P(r**2)*Ppe**2-Pppe,Q(r**2)/4)]).simplify()
O13 = O13*exp(I*(u*y-x*v))
O13

(-2*(P(r**2)**2 - 1)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) + (u**2 - 2*u*x + x**2)*P(r**2) - (v**2 - 2*v*y + y**2)*Q(r**2))*exp(I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

and $\Omega_{3,1}$

In [28]:
O20 = (Omega[2,0]*exp(I*(u*y-x*v))).simplify().expand().factor()
O20 = O20.collect(x*u).collect(x**2).collect(u**2)
O20 = O20.simplify()
O20 = O20.collect(-P(r**2)**2*Pppe+P(r**2)*Ppe**2+Pppe)
O20 = O20.collect(P(r**2))
O20 = O20.collect(Ppe)
O20 = O20.subs([(P(r**2)**2*Pppe-P(r**2)*Ppe**2-Pppe,Q(r**2)/4)]).simplify()
O20 = O20*exp(-I*(u*y-x*v))
O20

(-2*(P(r**2)**2 - 1)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - (u**2 - 2*u*x + x**2)*Q(r**2) + (v**2 - 2*v*y + y**2)*P(r**2))*exp(-I*(u*y - v*x))/((P(r**2) - 1)*(P(r**2) + 1))

we see that their product is real and calculates to

In [29]:
O1320re = (O13*O20)
O1320re.simplify()

(2*(P(r**2)**2 - 1)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - (u**2 - 2*u*x + x**2)*P(r**2) + (v**2 - 2*v*y + y**2)*Q(r**2))*(2*(P(r**2)**2 - 1)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) + (u**2 - 2*u*x + x**2)*Q(r**2) - (v**2 - 2*v*y + y**2)*P(r**2))/((P(r**2) - 1)**2*(P(r**2) + 1)**2)

Now $E=\Im \big[ \Omega_{1,2} \big] \Im \big[ \Omega_{3,4} \big]
-\frac{1}{2} \Re 
\Big[\Omega_{1,4}\Omega_{3,2}
- 
\Omega_{2,4}\Omega_{3,1}
\Big]$ calculates to

In [30]:
myE = O01im*O23im-(O0321re-O1320re)/2
myE = myE.subs([(((u-x)**2).expand(),r**2-(v-y)**2)])
myE = myE.subs([(((u-x)**2),r**2-(v-y)**2)])
myE = myE.subs([(Q(r**2),4*P(r**2)**2*Pppe-4*P(r**2)*Ppe**2-4*Pppe)]).expand()
myE = myE.simplify().collect(r)
myE

(r**4*(-4*P(r**2)**3*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) + 12*P(r**2)**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + 4*P(r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) + 4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2) + r**2*(8*P(r**2)**4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 8*P(r**2)**3*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**3 - 14*P(r**2)**3*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - 16*P(r**2)**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) + 8*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**3 + 14*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) + 8*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2)) + P(r**2)**6 + 4*P(r**2)**4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 - 8*P(r**2)**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 - 3*P(r**2

Here, we can further simplify the coefficients of $r^4$, $r^2$, and $r^0$

In [31]:
myEr4 = myE.expand().coeff(r,4).factor()
myEr4

-2*(P(r**2)**3*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 3*P(r**2)**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 - P(r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2)/((P(r**2) - 1)**2*(P(r**2) + 1)**2)

In [32]:
myEr2 = myE.expand().coeff(r,2).factor()
myEr2

(4*P(r**2)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 4*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 - 7*P(r**2) - 4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2))*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)/((P(r**2) - 1)*(P(r**2) + 1))

In [33]:
myEr0 = (myE-r**4*myEr4- r**2 * myEr2).factor()
myEr0

(P(r**2)**2 + 4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + 2)/2

We can now define $U = E/(1-P^2) -1$

In [34]:
myU = 1/(1-P(r**2)**2)*myE-1
myU = myU.simplify()
#myU = myU.factor()
myU

(4*r**4*(P(r**2)**3*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 3*P(r**2)**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 - P(r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2) + 2*r**2*(-4*P(r**2)**4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) + 4*P(r**2)**3*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + 7*P(r**2)**3 + 8*P(r**2)**2*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2) - 4*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 - 7*P(r**2) - 4*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2))*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) - 2*(P(r**2)**2 - 1)*(P(r**2)**4 - 2*P(r**2)**2 + 1) - P(r**2)**6 - 4*P(r**2)**4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + 8*P(r**2)**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 + 3*P(r**2)**2 - 4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2 - 2)/(2*(P(r**2)**2 - 1)*(P(r**2)**4 - 2*P(r**2)**2 + 1))

We define $I$, calculate the derivative, and show that it is equal to $U$

In [35]:
Pe = P(r**2)
myI = r**2*(2*Ppe**2+3*Pe**2/2)/(1-Pe**2) + 2*r**4*Pe*Ppe/(1-Pe**2)**2
myI = myI.subs([(r**2, s)])
myI

2*s**2*P(s)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, s)/(1 - P(s)**2)**2 + s*(3*P(s)**2/2 + 2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, s)**2)/(1 - P(s)**2)

In [36]:
dmyI = diff(myI,s)
dmyI = dmyI.subs([(diff(Ppe.subs([(r**2, s)]),s),Pppe),(diff(Pe.subs([(r**2, s)]),s),Ppe)])
dmyI = dmyI.subs([(s, r**2)])
dmyI

2*r**4*P(r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2)/(1 - P(r**2)**2)**2 + 2*r**4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2/(1 - P(r**2)**2)**2 + 8*r**4*P(r**2)**2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2/(1 - P(r**2)**2)**3 + r**2*(3*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2) + 4*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)*Subs(Derivative(P(_xi_1), (_xi_1, 2)), _xi_1, r**2))/(1 - P(r**2)**2) + 2*r**2*(3*P(r**2)**2/2 + 2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2)*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)/(1 - P(r**2)**2)**2 + 4*r**2*P(r**2)*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)/(1 - P(r**2)**2)**2 + (3*P(r**2)**2/2 + 2*Subs(Derivative(P(_xi_1), _xi_1), _xi_1, r**2)**2)/(1 - P(r**2)**2)

In [37]:
(myU-dmyI).simplify().expand()

0